In [1]:
import os
import json
import sys
import great_expectations as ge
import great_expectations.jupyter_ux
from datetime import datetime
import math
import pandas as pd
os.chdir('/Users/mparayil/Desktop/Development/dsa-data-workflows/grtexp_agero_dsa/great_expectations')

2020-03-17T11:17:39-0400 - INFO - Great Expectations logging enabled at INFO level by JupyterUX module.


In [2]:
import ge_prod.ge_data_access as gda
import ge_prod.queries as queries

In [3]:
rule_query = queries.queries.get('customer_experience').get('create_expectations_2019Q3-Q4')

In [4]:
rule_query

"SELECT * FROM customer_experience WHERE survey_time_utc >= to_date('2019-07-01')and survey_time_utc <= to_date('2019-12-31');"

# Author Expectations



[**Watch a short tutorial video**](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#video)

[**Read more in the tutorial**](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations)

**Reach out for help on** [**Great Expectations Slack**](https://tinyurl.com/great-expectations-slack)


### Get a DataContext object
[Read more in the tutorial](https://great-expectations.readthedocs.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#get-datacontext-object)




In [5]:
context = ge.data_context.DataContext()

2020-03-17T11:17:48-0400 - INFO - Using project config: /Users/mparayil/Desktop/Development/dsa-data-workflows/grtexp_agero_dsa/great_expectations/great_expectations.yml


### List data assets in your project

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#data-assets)


In [6]:
great_expectations.jupyter_ux.list_available_data_asset_names(context)

Inspecting your data sources. This may take a moment...


#### Pick one of the data asset names above and use as the value of data_asset_name argument below

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#get-batch)


### Specify data_asset & expectation_suite_name

In [7]:
data_asset_name = 'customer_experience'
normalized_data_asset_name = context.normalize_data_asset_name(data_asset_name)
print(normalized_data_asset_name)

NormalizedDataAssetName(datasource='agero_dsa_pandas', generator='default', generator_asset='customer_experience')


### Create a new empty expectation suite

In [8]:
expectation_suite_name = 'warnings_2019Q3-Q4'
# context.create_expectation_suite(data_asset_name=data_asset_name, expectation_suite_name=expectation_suite_name,
#                                 overwrite_existing=True)

In [9]:
context.list_expectation_suite_keys()

[{'data_asset_name': agero_dsa_pandas/default/service_specification,
  'expectation_suite_name': 'warnings_dec2019'},
 {'data_asset_name': agero_dsa_pandas/default/customer_experience,
  'expectation_suite_name': 'warnings_2019Q3-Q4'},
 {'data_asset_name': agero_dsa_pandas/default/network_outreach,
  'expectation_suite_name': 'warnings_2019Q4'},
 {'data_asset_name': agero_dsa_pandas/default/network_outreach,
  'expectation_suite_name': 'warnings_dec2019'},
 {'data_asset_name': agero_dsa_pandas/default/network_outreach,
  'expectation_suite_name': 'warnings'},
 {'data_asset_name': agero_dsa_pandas/default/customer_complaints,
  'expectation_suite_name': 'warnings_2019Q4'},
 {'data_asset_name': agero_dsa_pandas/default/network_claims,
  'expectation_suite_name': 'warnings_2019Q4'},
 {'data_asset_name': agero_dsa_pandas/default/network_claims,
  'expectation_suite_name': 'warnings_dec2019'},
 {'data_asset_name': agero_dsa_pandas/default/service_progress,
  'expectation_suite_name': 'warni

### Get batch to create expectations against

In [10]:
rule_query

"SELECT * FROM customer_experience WHERE survey_time_utc >= to_date('2019-07-01')and survey_time_utc <= to_date('2019-12-31');"

In [11]:
rule_df = gda.snowflake_connector_to_df(rule_query)
# rule_df.to_pickle('temp_data/network_claims_2019Q4.pkl')

In [12]:
# pickle_df = pd.read_pickle('temp_data/network_claims_2019Q4.pkl')

In [13]:
rule_df.shape

(795824, 6)

In [14]:
b_kwargs = {"dataset": rule_df}
batch = context.get_batch(normalized_data_asset_name, expectation_suite_name=expectation_suite_name,
                         batch_kwargs=b_kwargs)

In [15]:
batch.get_row_count()

795824

In [16]:
print(rule_df.shape)

(795824, 6)


In [17]:
[datasource['name'] for datasource in context.list_datasources() if datasource['class_name'] == 'PandasDatasource']

['agero_dsa_pandas']

In [18]:
# getting rule_df batchId & fingerprint
rule_batch_fingerprint = batch.batch_fingerprint
rule_batch_id = batch.batch_id

In [19]:
print('rule_batch_fingerprint: ', rule_batch_fingerprint, sep='\n')
print('rule_batch_id: ', rule_batch_id, sep='\n')

rule_batch_fingerprint: 
{'partition_id': '20200317T151849.768846Z', 'fingerprint': '9131d8c20ffb35fe759129fd42828dbb'}
rule_batch_id: 
{'timestamp': 1584458319.5815759, 'PandasInMemoryDF': True, 'fingerprint': 'ff48800a8b9a23eb331d78264d259c29'}


## Author Expectations

[Read more in the tutorial](https://docs.greatexpectations.io/en/latest/getting_started/create_expectations.html?utm_source=notebook&utm_medium=create_expectations#create-expectations)

See available expectations in the [expectation glossary](https://docs.greatexpectations.io/en/latest/glossary.html?utm_source=notebook&utm_medium=create_expectations)


### Dataset exploration & understanding of fields to ensure rules reflect behavior of data
- Validating columns to exist in table shape
- Expected column count in table shape
- Expected set values to be seen in given column
- Expected columns to have null or non-null values X percentage of the time
- Expect column values to be of certain data type(s)
- Placing max and min values limits on numerical columns
- Average or median column value to be within a certain range
- Expecting column A to be large/less than column B

### 1. Validating to see if every column exists in table

In [20]:
# add more expectations here
column_names = batch.get_table_columns()
print(column_names)

['CASE_ID', 'NPS_RESPONSE', 'SURVEY_COMMENTS', 'NPS_SOURCE', 'SURVEY_TIME_EASTERN', 'SURVEY_TIME_UTC']


In [23]:
batch.dtypes

CASE_ID                         int64
NPS_RESPONSE                     int8
SURVEY_COMMENTS                object
NPS_SOURCE                     object
SURVEY_TIME_EASTERN    datetime64[ns]
SURVEY_TIME_UTC        datetime64[ns]
dtype: object

In [22]:
# columns with multipe data types, will be changing them to int types in future
# case_id, po_number, vendor_id

In [23]:
master_column_names = ['CASE_ID', 'NPS_RESPONSE', 'SURVEY_COMMENTS', 'NPS_SOURCE', 'SURVEY_TIME_EASTERN', 
                       'SURVEY_TIME_UTC']

In [24]:
len(master_column_names)

6

In [25]:
len(column_names)

6

In [26]:
# Ensuring columns to exist
for col in master_column_names:
    print(col + ':', batch.expect_column_to_exist(col, result_format='BASIC', catch_exceptions=True), sep='\n')

CASE_ID:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
NPS_RESPONSE:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
SURVEY_COMMENTS:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
NPS_SOURCE:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
SURVEY_TIME_EASTERN:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
SURVEY_TIME_UTC:
{'success': True, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}


### 2. Validating column count in table is always the same

In [27]:
print('# of columns in network_claims: ', len(column_names))

# of columns in network_claims:  6


In [28]:
print('# of columns in {}: '.format('network_claims'), len(master_column_names), '\n')
if len(column_names) == len(master_column_names):
    print(batch.expect_table_column_count_to_equal(len(column_names), result_format='SUMMARY'))
else:
    print(batch.expect_table_column_count_to_equal(len(master_column_names), result_format='SUMMARY'))

# of columns in network_claims:  6 

{'success': True, 'result': {'observed_value': 6}}


### 3. Checking which columns should not have null values

In [26]:
# identifying which columns should not be null
print(column_names)

['CASE_ID', 'NPS_RESPONSE', 'SURVEY_COMMENTS', 'NPS_SOURCE', 'SURVEY_TIME_EASTERN', 'SURVEY_TIME_UTC']


In [27]:
rule_df.isnull().sum()

CASE_ID                     0
NPS_RESPONSE                0
SURVEY_COMMENTS        143652
NPS_SOURCE                  0
SURVEY_TIME_EASTERN         0
SURVEY_TIME_UTC             0
dtype: int64

In [29]:
# Separating null & non-null columns
null_cols = list(batch.isnull().sum()[batch.isnull().sum() > 0].keys())
not_null_cols = list(batch.isnull().sum()[batch.isnull().sum() == 0].keys())

In [30]:
not_null_cols

['CASE_ID',
 'NPS_RESPONSE',
 'NPS_SOURCE',
 'SURVEY_TIME_EASTERN',
 'SURVEY_TIME_UTC']

In [31]:
null_cols

['SURVEY_COMMENTS']

In [32]:
# checking for all columns that shouldn't be null are not
for col in not_null_cols:
    print(col, '\n', batch.expect_column_values_to_not_be_null(col, result_format='BASIC'))

CASE_ID 
 {'success': True, 'result': {'element_count': 795825, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
NPS_RESPONSE 
 {'success': True, 'result': {'element_count': 795825, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
NPS_SOURCE 
 {'success': True, 'result': {'element_count': 795825, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
SURVEY_TIME_EASTERN 
 {'success': True, 'result': {'element_count': 795825, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}
SURVEY_TIME_UTC 
 {'success': True, 'result': {'element_count': 795825, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': []}}


In [33]:
print(f"capturing the following columns to not be null: \n {not_null_cols}")

capturing the following columns to not be null: 
 ['CASE_ID', 'NPS_RESPONSE', 'NPS_SOURCE', 'SURVEY_TIME_EASTERN', 'SURVEY_TIME_UTC']


In [34]:
null_cols

['SURVEY_COMMENTS']

In [35]:
len(null_cols)

1

### 4. Validating columns to have null values
- **columns to check:**
    - SURVEY_COMMENTS

In [36]:
143652 / 163356

0.8793800044075516

In [37]:
rule_df.isnull().sum()

CASE_ID                     0
NPS_RESPONSE                0
SURVEY_COMMENTS        674821
NPS_SOURCE                  0
SURVEY_TIME_EASTERN         0
SURVEY_TIME_UTC             0
dtype: int64

In [60]:
w
print(weight)

0.1442552061601487


In [75]:
not_null_weights

{'SURVEY_COMMENTS': 0.111}

In [42]:
gda.get_df_not_null_weights()

0.9057

In [58]:
df_group = rule_df.groupby(rule_df['SURVEY_TIME_UTC'].dt.day)
df_group2 = df_group.apply(lambda x: x['SURVEY_COMMENTS'].notnull().mean())

weight = df_group2.quantile(0.1, 'midpoint')
final_w = (weight - 0.035).round(4)
final_w

0.1093

In [17]:
rule_df.isnull().sum()

CASE_ID                     0
NPS_RESPONSE                0
SURVEY_COMMENTS        674716
NPS_SOURCE                  0
SURVEY_TIME_EASTERN         0
SURVEY_TIME_UTC             0
dtype: int64

In [23]:
null_percents = (rule_df.isnull().sum() / len(rule_df))[rule_df.isnull().sum() / len(rule_df) > 0]
print(null_percents)
print('---------------------------------------')
for x, y in null_percents.iteritems():
    print(x, round(y,3))

SURVEY_COMMENTS    0.848406
dtype: float64
---------------------------------------
SURVEY_COMMENTS 0.848


In [25]:
# calculating weight for columns of how often they should be null
null_percents = (1 - (batch.isnull().sum() / len(batch))[batch.isnull().sum() / len(batch) > 0])
for x, y in null_percents.items():
    print(x, y, sep='\n')
    
not_null_weights = dict(null_percents)

# lowering weights by one thousandth of decimal
for key, original_weight in not_null_weights.items():
    not_null_weights[key] = round((original_weight - 0.01), 3)
    
print('---------------------------------------')
print('not null weights:')
for x, y in not_null_weights.items():
    print(x,y)

SURVEY_COMMENTS
0.15159410266888818
---------------------------------------
not null weights:
SURVEY_COMMENTS 0.142


In [19]:
# validating columns that should be null with 
w = gda.get_df_not_null_weights(batch, 'SURVEY_TIME_UTC', 'SURVEY_COMMENTS')
# print('SURVEY_COMMENTS', batch.expect_column_values_to_not_be_null('SURVEY_COMMENTS', mostly=w, 
#                                                                    catch_exceptions=True, 
#                                                                    result_format='SUMMARY'), sep='\n')

In [20]:
w

0.0718

### 5. Expecting column values to be in a set
- nps_source
- nps_response

In [11]:
null_percents = (rule_df.isnull().sum() / len(rule_df))[rule_df.isnull().sum() / len(rule_df) > 0]
print(null_percents)
print('---------------------------------------')
for x, y in null_percents.iteritems():
    print(x, round(y,3))

NameError: name 'rule_df' is not defined

In [81]:
rule_df.dtypes

CASE_ID                         int64
NPS_RESPONSE                   object
SURVEY_COMMENTS                object
NPS_SOURCE                     object
SURVEY_TIME_EASTERN    datetime64[ns]
SURVEY_TIME_UTC        datetime64[ns]
dtype: object

In [63]:
category_cols = ['NPS_SOURCE', 'NPS_RESPONSE']

In [64]:
for col in category_cols:
    print(list(batch.get_column_value_counts(col, sort='value').keys()), '\n')

['sms', 'web'] 

['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9'] 



In [65]:
print(batch.get_column_value_counts('NPS_RESPONSE', sort='value'))

value
0      26071
1       5285
10    616387
2       4945
3       4681
4       4827
5      11137
6       6844
7      16770
8      42036
9      56842
Name: count, dtype: int64


In [67]:
val_set_list = []

for col in category_cols:
    list_sets = list(batch.get_column_value_counts(col, sort='value').keys())
    val_set_list.append(list_sets)

In [68]:
val_set_list

[['sms', 'web'], ['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9']]

In [69]:
for col, x in zip(category_cols,val_set_list):
    print(col, '\n', x, '\n')

NPS_SOURCE 
 ['sms', 'web'] 

NPS_RESPONSE 
 ['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9'] 



In [70]:
for col, vals in zip(category_cols, val_set_list):
    print(col, '\n', batch.expect_column_values_to_be_in_set(col, vals, result_format='BASIC', 
                                        include_config=True, catch_exceptions=True), '\n')

NPS_SOURCE 
 {'success': True, 'result': {'element_count': 795825, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'unexpected_percent_nonmissing': 0.0, 'partial_unexpected_list': []}, 'expectation_config': {'expectation_type': 'expect_column_values_to_be_in_set', 'kwargs': {'column': 'NPS_SOURCE', 'value_set': ['sms', 'web'], 'result_format': 'BASIC'}}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}} 

NPS_RESPONSE 
 {'success': True, 'result': {'element_count': 795825, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'unexpected_percent_nonmissing': 0.0, 'partial_unexpected_list': []}, 'expectation_config': {'expectation_type': 'expect_column_values_to_be_in_set', 'kwargs': {'column': 'NPS_RESPONSE', 'value_set': ['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9'], 'result_format': 'BASIC'}}, 'exception_info': {'raised_exception': Fals

### 8. Determine if columns are unique per row
- CASE_ID

In [79]:
1 - ((rule_df.shape[0] - len(rule_df['CASE_ID'].unique())) / rule_df.shape[0])

0.9988188357993277

In [78]:
# TODO: verify if unique, according to notes from Austin & Jody it is. 
batch.expect_column_values_to_be_unique('CASE_ID', mostly=.99, result_format='SUMMARY',
                                              catch_exceptions=True)

{'success': True,
 'result': {'element_count': 795825,
  'missing_count': 0,
  'missing_percent': 0.0,
  'unexpected_count': 1832,
  'unexpected_percent': 0.23020136336506142,
  'unexpected_percent_nonmissing': 0.23020136336506142,
  'partial_unexpected_list': [2280652458,
   50283598296,
   0,
   2280670983,
   50283598295,
   50283598226,
   50283598292,
   50283598169,
   2280673061,
   50283598293,
   2280594635,
   50283598290,
   50283598291,
   2280635152,
   2280647560,
   2280235026,
   2280567550,
   50283598289,
   50283598231,
   50283597986],
  'partial_unexpected_index_list': [5,
   18,
   40,
   43,
   45,
   57,
   131,
   221,
   249,
   294,
   302,
   329,
   397,
   995,
   1150,
   1160,
   1164,
   1168,
   1419,
   1486],
  'partial_unexpected_counts': [{'value': 0, 'count': 14},
   {'value': 2288885951, 'count': 4},
   {'value': 50283621359, 'count': 4},
   {'value': 2278237146, 'count': 3},
   {'value': 2281150269, 'count': 3},
   {'value': 2281980620, 'count':

### 9. Looking at column A to be greater than column B
- SURVEY_TIME_UTC > SURVEY_TIME_EASTERN

In [80]:
batch.expect_column_pair_values_A_to_be_greater_than_B('SURVEY_TIME_UTC', 'SURVEY_TIME_EASTERN', 
                                                       ignore_row_if='either_value_is_missing', 
                                                       result_format='SUMMARY', catch_exceptions=True)

{'success': True,
 'result': {'element_count': 795825,
  'missing_count': 0,
  'missing_percent': 0.0,
  'unexpected_count': 0,
  'unexpected_percent': 0.0,
  'unexpected_percent_nonmissing': 0.0,
  'partial_unexpected_list': [],
  'partial_unexpected_index_list': [],
  'partial_unexpected_counts': []},
 'exception_info': {'raised_exception': False,
  'exception_message': None,
  'exception_traceback': None}}

### 8. Expecting columns to be certain data type

In [81]:
rule_df.dtypes

CASE_ID                         int64
NPS_RESPONSE                   object
SURVEY_COMMENTS                object
NPS_SOURCE                     object
SURVEY_TIME_EASTERN    datetime64[ns]
SURVEY_TIME_UTC        datetime64[ns]
dtype: object

In [82]:
for x, y in batch.dtypes.iteritems():
    print(x, y)

CASE_ID int64
NPS_RESPONSE object
SURVEY_COMMENTS object
NPS_SOURCE object
SURVEY_TIME_EASTERN datetime64[ns]
SURVEY_TIME_UTC datetime64[ns]


In [83]:
customer_experience_data_types = dict(batch.dtypes.iteritems())

In [84]:
customer_experience_data_types

{'CASE_ID': dtype('int64'),
 'NPS_RESPONSE': dtype('O'),
 'SURVEY_COMMENTS': dtype('O'),
 'NPS_SOURCE': dtype('O'),
 'SURVEY_TIME_EASTERN': dtype('<M8[ns]'),
 'SURVEY_TIME_UTC': dtype('<M8[ns]')}

In [85]:
for key, val in customer_experience_data_types.items():
    customer_experience_data_types[key] = str(val)

In [86]:
customer_experience_data_types

{'CASE_ID': 'int64',
 'NPS_RESPONSE': 'object',
 'SURVEY_COMMENTS': 'object',
 'NPS_SOURCE': 'object',
 'SURVEY_TIME_EASTERN': 'datetime64[ns]',
 'SURVEY_TIME_UTC': 'datetime64[ns]'}

In [87]:
for col, typ in customer_experience_data_types.items():
    print(batch.expect_column_values_to_be_of_type(col, typ, result_format='SUMMARY', catch_exceptions=True))

{'success': True, 'result': {'observed_value': 'int64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'object_'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'object_'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'object_'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'datetime64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}
{'success': True, 'result': {'observed_value': 'datetime64'}, 'exception_info': {'raised_exception': False, 'exception_message': None, 'exception_traceback': None}}


In [94]:
batch.NPS_RESPONSE = batch.NPS_RESPONSE.astype(str).astype(int)

In [290]:
dfg = rule_df.groupby(rule_df["SURVEY_TIME_UTC"].dt.date)
dfg2 = dfg.apply(lambda x: x['NPS_RESPONSE'].mean())

In [291]:
dfg2.quantile([ 0., .05, .1, .33,  .667 ,1.0 ])

0.000    8.906498
0.050    9.000429
0.100    9.044860
0.330    9.120880
0.667    9.183387
1.000    9.301079
dtype: float64

In [258]:
# quantile_ranges= {
#     "value_ranges": [[0,8.99], [9.0, 9.7], [9.71, 9.91], [9.92, 10.0]],
#     "quantiles": [ 0. ,  .33,  .667 , 1.0 ]}

batch.expect_column_mean_to_be_between('NPS_RESPONSE', min_value=8.9, max_value=9.3, result_format='COMPLETE')

In [177]:
"value_ranges" = [(1,2), (3,4), (5,6), (7,8), (9,10)]

SyntaxError: can't assign to literal (<ipython-input-177-dadbbcbad233>, line 1)

In [163]:
batch.get_column_stdev('NPS_RESPONSE')

2.2225126852302513

In [169]:
batch.get_column_partition('NPS_RESPONSE', ([ 0. ,  2.5,  5. ,  7.5, 10. ]))

ValueError: Invalid parameter for bins argument

In [128]:
batch.get_column_mean('NPS_RESPONSE').DS_Store

9.140704300568592

In [162]:
batch.get_column_quantiles('NPS_RESPONSE', quantiles=(0.0, .25, .5, .75))

[0, 10, 10, 10]

In [168]:
batch.get_column_partition('NPS_RESPONSE', bins='uniform', n_bins=4)

array([ 0. ,  2.5,  5. ,  7.5, 10. ])

In [116]:
batch.get_column_hist('NPS_RESPONSE', ([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]))

[26071, 5285, 4945, 4681, 4827, 11137, 6844, 16770, 42036, 673229]

In [112]:
batch.get_row_count()

795825

In [194]:
# partition_obj = ge.dataset.util.categorical_partition_data(batch.NPS_RESPONSE)

# batch.expect_column_chisquare_test_p_value_to_be_greater_than("NPS_RESPONSE", partition_object=partition_obj, 
#                                                              result_format='SUMMARY')

### Review the expectations

Expectations that were true on this data batch were added. To view all the expectations you added so far about this data asset, do:

In [200]:
batch.get_expectation_suite()

2020-02-25T01:48:20-0500 - INFO - 	24 expectation(s) included in expectation_suite. result_format settings filtered.


{'data_asset_name': 'agero_dsa_pandas/default/customer_experience',
 'expectation_suite_name': 'warnings_dec2019',
 'meta': {'great_expectations.__version__': '0.8.8'},
 'expectations': [{'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'CASE_ID'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'NPS_RESPONSE'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'SURVEY_COMMENTS'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'NPS_SOURCE'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'SURVEY_TIME_EASTERN'}},
  {'expectation_type': 'expect_column_to_exist',
   'kwargs': {'column': 'SURVEY_TIME_UTC'}},
  {'expectation_type': 'expect_table_column_count_to_equal',
   'kwargs': {'value': 6}},
  {'expectation_type': 'expect_column_values_to_not_be_null',
   'kwargs': {'column': 'CASE_ID'}},
  {'expectation_type': 'expect_column_values_to_not_be_null',
   'kwargs': 

In [201]:
batch.save_expectation_suite()

2020-02-25T01:48:31-0500 - INFO - 	24 expectation(s) included in expectation_suite. result_format settings filtered.


### You created and saved expectations for at least one of the data assets.

### We will show you how to set up validation - the process of checking if new files of this type conform to your expectations before they are processed by your pipeline's code. 

### Go to [integrate_validation_into_pipeline.ipynb](integrate_validation_into_pipeline.ipynb) to proceed.


